In [10]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

Import Data

In [11]:
def to_onehot(x):
    labels = np.unique(x)
    result = np.zeros(shape=(x.shape[0], labels.shape[0]))

    for i in range(len(x)):
        result[i][x[i]] = 1.0

    return result

In [12]:
from sklearn.model_selection import train_test_split

X = [] # Features per class
y = [] # Labels
N = 5250 # Amount of data we want to use max: 5250

# Import the features
with open("traindata.txt", "r") as file:
    for line in file.readlines()[:N]:
        features = [float(i) for i in line.split(",")]
        X.append(features)

# Import the labels
with open("trainlabels.txt", "r") as file:
    for line in file.readlines()[:N]:
        label = float(line.rstrip())
        y.append(label)
    
# Convert data to numpy arrays
X = np.array(X)
y = np.array(y, dtype=np.int32)
y_onehot = to_onehot(y)

def split_data(X, y, test_size=0.2, val_size=0.2, random_state=42):
   
    # Splitting the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Further splitting the training data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size / (1 - test_size),
                                                      random_state=random_state)

    return X_train, X_val, X_test, y_train, y_val, y_test

X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y_onehot)

Preprocess Data

In [13]:
# Calculates the number of components to consider when performing pca
def num_components(X, variance_tol = 0.8):
    # Standardize each feature of the matrix
    x_mean = np.mean(X, axis = 0)
    x_std = np.std(X, axis = 0)
    Z = (X - x_mean) / x_std

    # Calculate covariance matrix
    C = np.cov(Z, rowvar=False)
    # Calculate eigenvalues and eigenvectors and sort by size
    eigenvalues, eigenvectors = np.linalg.eig(C)
    index = eigenvalues.argsort()[:: -1]
    eigenvalues = eigenvalues[index]
    eigenvectors = eigenvectors[:, index]

    # Calculate explained variance matrix 
    explained_var = np.cumsum(eigenvalues) / np.sum(eigenvalues)

    # Select number of components responsible for variance_tol% of variance
    n_components = np.argmax(explained_var >= variance_tol) + 1
    return Z, x_mean, x_std, n_components

# Parameters are trained components, trained mean, trained standard deviation and the new inputs X
# Changes to the PCA basis
def convert_to_pca(components, mean, std, X):
    Z = (X - mean)/std
    return np.transpose(np.matmul(components, np.transpose(Z)))

Z, mean, std, n_components = num_components(X_train, 0.8)
# Initialize prinicipal component analysis
pca = PCA(n_components)
pca.fit(Z)
components = pca.components_
X_train = pca.transform(Z)
temp = pca.transform(X_test)
X_test = convert_to_pca(components, mean, std, X_test)
X_val = convert_to_pca(components, mean, std, X_val)


(3150, 1041)


Build Model

In [14]:
import torch as T
import torch.nn as nn


# Build Model
def build_model(n_inputs, n_outputs, h1_dims=512, h2_dims=256):
    model =  nn.Sequential(
        nn.Linear(n_inputs, h1_dims),
        nn.ReLU(),
        nn.Linear(h1_dims, h2_dims),
        nn.ReLU(),                              
        nn.Linear(h2_dims, n_outputs),
        nn.Softmax(dim=-1)
    )
    return model


class NeuralNetwork(nn.Module):
    def __init__(self, n_inputs, n_outputs, h1_dims=512, h2_dims=256, name="NeuralNetwork", save_dir="/trained_models"):
        super(NeuralNetwork, self).__init__()
        self.name = name
        self.save_dir = save_dir

        self.model = build_model(n_inputs, n_outputs, h1_dims, h2_dims)
    
    def forward(self, X):
        X = self.model(X)
        return X
    
    def save_model(self):
        T.save(self.state_dict, f"{self.save_dir}/{self.name}.pth")

    def load_model(self, name):
        self.load_state_dict(T.load(f"{self.save_dir}/{name}.pth"))



labels = np.unique(y) # Labels of our data (0 - 20)

n_inputs = X_train.shape[1]
h1_dims = 2 * n_inputs # Number of nodes for the 1st hidden layer
h2_dims = n_inputs # Number of nodes for the 2nd hidden layer
n_outputs = labels.shape[0] # 21 labels

# Initialize the model
net = NeuralNetwork(n_inputs=n_inputs, n_outputs=n_outputs, h1_dims=h1_dims, h2_dims=h2_dims, name="NN-v1")


Train Model

In [15]:
import torch.optim as optim

# Global Variables
epochs = 10
learning_rate = 1e-5

loss_func = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)


def test_model(model, X_test, y_test, loss_func):
    size = X_test.shape[1]

    model.eval()
    with T.no_grad():
        X = T.from_numpy(X_test).to(T.float32)
        y_tensor = T.Tensor(y_test).to(T.float)

        y_pred = model.forward(X)

        test_loss = loss_func(y_pred, y_tensor)

        correct = (y_pred.argmax(1) == y_tensor.argmax(1)).type(T.float).sum().item()
        
        test_loss /= size
        correct /= size
        print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


def train_model(model, X_train, y_train, loss_func, optimizer):
    size = len(X_train)
    batch_size = 50

    for i in range(size//batch_size):
        start = batch_size * i
        end = start + batch_size
        X = T.from_numpy(X_train[start:end]).to(T.float32)
        y_true = T.Tensor(y_train[start:end]).to(T.float)

        y_pred = model(X)
        loss = loss_func(y_pred, y_true)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i * batch_size) % 250 == 0:
            loss, current = loss.item(), (i + 1) * batch_size
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")

    train_model(net, X_train, y_train, loss_func, optimizer)
    print('Finished training')
    test_model(net, X_test, y_test, loss_func)




Epoch 1
-------------------------------
loss: 0.045372  [   50/ 3150]
loss: 0.045574  [  300/ 3150]
loss: 0.045313  [  550/ 3150]
loss: 0.045334  [  800/ 3150]
loss: 0.045585  [ 1050/ 3150]
loss: 0.045630  [ 1300/ 3150]
loss: 0.045541  [ 1550/ 3150]
loss: 0.045348  [ 1800/ 3150]
loss: 0.045090  [ 2050/ 3150]
loss: 0.045493  [ 2300/ 3150]
loss: 0.045317  [ 2550/ 3150]
loss: 0.045475  [ 2800/ 3150]
loss: 0.045251  [ 3050/ 3150]
Finished training
Test Error: 
 Accuracy: 33.8%, Avg loss: 0.000334 

Epoch 2
-------------------------------
loss: 0.045265  [   50/ 3150]
loss: 0.045510  [  300/ 3150]
loss: 0.045261  [  550/ 3150]
loss: 0.045264  [  800/ 3150]
loss: 0.045506  [ 1050/ 3150]
loss: 0.045578  [ 1300/ 3150]
loss: 0.045483  [ 1550/ 3150]
loss: 0.045296  [ 1800/ 3150]
loss: 0.045012  [ 2050/ 3150]
loss: 0.045436  [ 2300/ 3150]
loss: 0.045257  [ 2550/ 3150]
loss: 0.045414  [ 2800/ 3150]
loss: 0.045214  [ 3050/ 3150]
Finished training
Test Error: 
 Accuracy: 37.5%, Avg loss: 0.000333 

